### Charger les embeddings GloVe et FastText
- Les liens pour télécharger les embeddings GloVe et FastText sont les suivants :
- https://nlp.stanford.edu/projects/glove/
- https://fasttext.cc/docs/en/english-vectors.html


## Développement du "Modèle sur mesure avancé" avec des essais sur au moins deux word embeddings différents et en gardant celui qui permet d’obtenir les meilleures performances. Nous utiliserons Glove et FastText.
-  Nous allons construire et entraîner un réseau de neurones pour chaque embedding, évaluer les performances et enregistrer les expérimentations avec MLFlow. 
-  C'est la colonne clean_text_embeddings qui sera utilisée pour les embeddings.
- Par rapport au notebook précédent, nous allons enregistrer des métriques supplémentaires, visualisation des courbes ROC/AUC, et optimisation des hyperparamètres, à intégrer dans le notebook, en gardant le sous-échantillonnage initial pour réduire la charge de calcul.

### Charger le fichier CSV nettoyé

### Cellule 1 : Vérification de la version de TensorFlow

In [21]:
# Verification version de tensorflow
import tensorflow as tf
print(tf.__version__)

2.17.0


### Cellule 2 : Chargement des données nettoyées

In [22]:
import os
import pandas as pd

# Chemin relatif pour charger les données nettoyées
file_path = os.path.join("..", "data", "cleaned_data_with_text_for_models.csv")

# Chargement du DataFrame nettoyé
data = pd.read_csv(file_path, index_col=0)

# Vérification du contenu du DataFrame

### Cellule 3 : Préparation des données

In [23]:
# Réinitialiser l'index pour récupérer toutes les colonnes, y compris 'text' si elle est utilisée comme index
data = data.reset_index()

# Vérifier les colonnes disponibles après réinitialisation de l'index
print(data.columns)

# Dataframe info
data.info()


Index(['text', 'clean_text_tfidf', 'clean_text_embeddings', 'clean_text_bert',
       'target'],
      dtype='object')


### Cellule 4 : Traitement des valeurs manquantes

In [ ]:
# Vérifier les valeurs manquantes
print(data.isnull().sum())

# Filtrer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
missing_embeddings = data[data['clean_text_embeddings'].isnull()]
print(missing_embeddings[['text', 'clean_text_tfidf', 'clean_text_embeddings']].head(2))

# Supprimer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
data = data.dropna(subset=['clean_text_embeddings'])

# Vérifier les valeurs manquantes après suppression
print(data.isnull().sum())


### Cellule 5 : Chargement des embeddings GloVe et FastText

In [ ]:
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

# Chemin relatif pour charger les fichiers d'embeddings
glove_path = os.path.join("..", "data", "glove.twitter.27B.100d.txt")
fasttext_path = os.path.join("..", "data", "crawl-300d-2M-subword.bin")

# Charger les embeddings GloVe
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Charger les embeddings FastText (format Facebook binaire)
fasttext_model = load_facebook_vectors(fasttext_path)


### Cellule 6 : Création de la matrice d'embeddings

In [ ]:
import numpy as np

def create_embedding_matrix(embedding_model, vocab, embedding_dim):
    embedding_matrix = np.zeros((len(vocab) + 1, embedding_dim))
    for word, i in vocab.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
        else:
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix


### Cellule 7 : Préparation des données pour l'entraînement

In [ ]:
from sklearn.model_selection import train_test_split

# sous-échantillonnage pour réduire la charge de calcul à 0.1% pour tester rapidement le pipeline
data = data.sample(frac=0.001, random_state=42)

# Tokenisation des textes nettoyés
num_words = 10000
max_sequence_length = 100


# Créer une couche TextVectorization
tv_layer = tf.keras.layers.TextVectorization(
    max_tokens=num_words,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapter TextVectorization sur les textes
tv_layer.adapt(data['clean_text_embeddings'])

# Convertir les textes en séquences
X = tv_layer(data['clean_text_embeddings']).numpy()
y = data['target'].values

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Nouvelle Cellule 8 : Création de la matrice d'embeddings pour GloVe

In [ ]:
# Taille des vecteurs d'embeddings pour GloVe
embedding_dim_glove = 100  # Assurez-vous que cette taille correspond à celle de votre fichier GloVe

# Créer la matrice d'embeddings pour GloVe
embedding_matrix_glove = create_embedding_matrix(glove_model, tv_layer.get_vocabulary(), embedding_dim_glove)

# Vérifier la forme de la matrice d'embeddings pour s'assurer qu'elle est correctement initialisée
print(f"Shape of GloVe embedding matrix: {embedding_matrix_glove.shape}")


### Nouvelle Cellule 9 : Création de la matrice d'embeddings pour FastText

In [ ]:
# Taille des vecteurs d'embeddings pour FastText
embedding_dim_fasttext = 300  # Assurez-vous que cette taille correspond à celle de votre fichier FastText

# Créer la matrice d'embeddings pour FastText
embedding_matrix_fasttext = create_embedding_matrix(fasttext_model, tv_layer.get_vocabulary(), embedding_dim_fasttext)

# Vérifier la forme de la matrice d'embeddings pour s'assurer qu'elle est correctement initialisée
print(f"Shape of FastText embedding matrix: {embedding_matrix_fasttext.shape}")

### Cellule 10 : Définition du modèle de deep learning

In [ ]:
def build_model(embedding_matrix, input_length):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=embedding_matrix.shape[0],
                                        output_dim=embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=input_length,
                                        trainable=False))
    model.add(tf.keras.layers.LSTM(128, return_sequences=False))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


### Cellule 11 : Intégration du Grid Search avec early stopping   


In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def build_model_for_search(embedding_matrix, input_length):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=embedding_matrix.shape[0],
                                        output_dim=embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=input_length,
                                        trainable=False))
    model.add(tf.keras.layers.LSTM(128, return_sequences=False))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Paramètres à tester
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [5, 10]
}

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Utiliser KerasClassifier pour intégrer le modèle dans scikit-learn
model_glove_for_search = KerasClassifier(build_fn=lambda: build_model_for_search(embedding_matrix_glove, input_length=100))

grid = GridSearchCV(estimator=model_glove_for_search, param_grid=param_grid, cv=3)

# Adapter le GridSearch sur les données
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Afficher les meilleurs paramètres trouvés
print(f"Meilleurs hyperparamètres : {grid_result.best_params_}")

### Cellule 12 : Entraînement du modèle GloVe avec les meilleurs hyperparamètres

In [ ]:
# Entraîner le modèle avec les meilleurs paramètres trouvés
best_batch_size = grid_result.best_params_['batch_size']
best_epochs = grid_result.best_params_['epochs']

model_glove = build_model(embedding_matrix_glove, input_length=100)
history_glove = model_glove.fit(X_train, y_train, epochs=best_epochs, batch_size=best_batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])

### Cellule 13 : Définition de la fonction evaluate_model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

def evaluate_model(model, X_test, y_test, model_name):
    # Prédictions du modèle
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    y_pred_proba = model.predict(X_test)
    
    # Calcul des métriques
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Enregistrement des métriques dans MLFlow
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)

    # Tracer la courbe ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, marker='.')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {model_name}')
    roc_curve_path = f"roc_curve_{model_name}.png"
    plt.savefig(roc_curve_path)
    mlflow.log_artifact(roc_curve_path)
    plt.close()

    print(f"Model evaluation for {model_name} completed.")


### Cellule 14 : Évaluation et enregistrement du modèle pour GloVe

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns"
mlruns_path = Path("../mlruns").resolve()

# Vérifier que le dossier "mlruns" existe, sinon le créer
if not mlruns_path.exists():
    mlruns_path.mkdir(parents=True)

# Vérifier que le sous-dossier ".trash" existe, sinon le créer
trash_folder = mlruns_path / ".trash"
if not trash_folder.exists():
    trash_folder.mkdir(parents=True)

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Créer ou utiliser l'expérience existante
experiment_name = "GloVe_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour GloVe
with mlflow.start_run(nested=True):
    mlflow.log_param("embedding", "GloVe")
    mlflow.log_param("embedding_dim", embedding_dim_glove)
    mlflow.log_param("batch_size", best_batch_size)
    mlflow.log_param("epochs", best_epochs)

    # Évaluer le modèle et enregistrer les métriques et la courbe ROC
    evaluate_model(model_glove, X_test, y_test, "GloVe")
    mlflow.keras.log_model(model_glove, "model_glove")

print("Modèle GloVe évalué et enregistré dans MLFlow.")


### Cellule 15 : Grid Search pour le modèle FastText avec early stopping

In [ ]:
# Utiliser KerasClassifier pour intégrer le modèle FastText dans scikit-learn
model_fasttext_for_search = KerasClassifier(build_fn=lambda: build_model_for_search(embedding_matrix_fasttext, input_length=100))

# Définition des paramètres à tester pour FastText
param_grid_fasttext = {
    'batch_size': [16, 32, 64],
    'epochs': [5, 10]
}

# Configurer le GridSearchCV pour le modèle FastText
grid_fasttext = GridSearchCV(estimator=model_fasttext_for_search, param_grid=param_grid_fasttext, cv=3)

# Adapter le GridSearch sur les données
grid_result_fasttext = grid_fasttext.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Afficher les meilleurs paramètres trouvés pour FastText
print(f"Meilleurs hyperparamètres pour FastText : {grid_result_fasttext.best_params_}")

### Cellule 16 : Entraînement du modèle FastText avec les meilleurs hyperparamètres

In [ ]:
best_batch_size_fasttext = grid_result_fasttext.best_params_['batch_size']
best_epochs_fasttext = grid_result_fasttext.best_params_['epochs']

model_fasttext = build_model(embedding_matrix_fasttext, input_length=100)

history_fasttext = model_fasttext.fit(
    X_train,
    y_train,
    epochs=best_epochs_fasttext,
    batch_size=best_batch_size_fasttext,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)


### Cellule 17 : Évaluation et enregistrement du modèle FastText avec MLFlow

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns"
mlruns_path = Path("../mlruns").resolve()

# Vérifier que le dossier "mlruns" existe, sinon le créer
if not mlruns_path.exists():
    mlruns_path.mkdir(parents=True)

# Vérifier que le sous-dossier ".trash" existe, sinon le créer
trash_folder = mlruns_path / ".trash"
if not trash_folder.exists():
    trash_folder.mkdir(parents=True)

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Créer ou utiliser l'expérience existante pour FastText
experiment_name_fasttext = "FastText_Embedding_Experiment"
mlflow.set_experiment(experiment_name_fasttext)

# Démarrer une nouvelle session MLFlow pour FastText
with mlflow.start_run(nested=True):
    mlflow.log_param("embedding", "FastText")
    mlflow.log_param("embedding_dim", embedding_dim_fasttext)
    mlflow.log_param("batch_size", best_batch_size_fasttext)
    mlflow.log_param("epochs", best_epochs_fasttext)

    # Évaluer le modèle FastText et enregistrer les métriques et la courbe ROC
    evaluate_model(model_fasttext, X_test, y_test, "FastText")
    mlflow.keras.log_model(model_fasttext, "model_fasttext")

print("Modèle FastText évalué et enregistré dans MLFlow.")



### 18. Conclusion
- Nous avons construit et entraîné des modèles de deep learning pour la classification de tweets en utilisant les embeddings GloVe et FastText.
- Les performances des modèles ont été évaluées et enregistrées avec MLFlow.
- Les hyperparamètres ont été optimisés avec Grid Search et Early Stopping contrairement au notebook précédent.